<a href="https://colab.research.google.com/github/MichaelKru92/MASTER/blob/main/Michael_Kruszewski_von_Woche_2_import_csv_to_db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Github Repository klonen
Damit Sie die CSV-Dateien direkt aus dem angegebenen GitHub-Repository laden können, klonen Sie das Repository in Ihrer Colab-Umgebung.

Danach sind alle CSV-Dateien (allergies.csv, careplans.csv, conditions.csv, devices.csv, encounters.csv, imaging_studies.csv, immunizations.csv, medications.csv) lokal verfügbar (im Ordner synthea_data/).


In [30]:
!git clone https://github.com/gaetanwabo/synthea_data.git


fatal: destination path 'synthea_data' already exists and is not an empty directory.


# 2. Tabellenstruktur und Datenbankerstellung
In den nächsten Schritten wird eine neue SQLite-Datenbank angelegt und alle benötigten Tabellen erstellt. Die SQL-Befehle basieren auf den vorgegebenen Tabellenstrukturen

In [31]:
import sqlite3
import csv
import os

# Legen Sie eine neue SQLite-Datenbank an (bzw. verbinden sich mit einer bestehenden).
conn = sqlite3.connect("synthea.db")
cursor = conn.cursor()

# ----------------------------------------------------------------------------
# 1) Definition aller Tabellen per SQL
# ----------------------------------------------------------------------------

sql_table = {}

sql_table["patients"] = """CREATE TABLE IF NOT EXISTS patients (
    Id STRING PRIMARY KEY,
    BIRTHDATE DATE,
    DEATHDATE DATE,
    SSN STRING,
    DRIVERS STRING,
    PASSPORT STRING,
    PREFIX STRING,
    FIRST STRING,
    LAST STRING,
    SUFFIX STRING,
    MAIDEN STRING,
    MARITAL STRING,
    RACE STRING,
    ETHNICITY STRING,
    GENDER STRING,
    BIRTHPLACE STRING,
    ADDRESS STRING,
    CITY STRING,
    STATE STRING,
    COUNTRY STRING,
    ZIP STRING,
    LAT INTEGER,
    LON INTEGER,
    HEALTHCARE_EXPENSES INTEGER,
    HEALTHCARE_COVERAGE INTEGER
);"""

sql_table["encounters"] = """CREATE TABLE IF NOT EXISTS encounters (
    Id STRING PRIMARY KEY,
    START DATE,
    STOP DATE,
    PATIENT STRING,
    ORGANIZATION STRING,  -- statt ORGANIZATIONS
    PROVIDER STRING,
    PAYER STRING,
    ENCOUNTERCLASS STRING,
    CODE STRING,
    DESCRIPTION STRING,
    BASE_ENCOUNTER_COST INTEGER,
    TOTAL_CLAIM_COST INTEGER,
    PAYER_COVERAGE INTEGER,
    REASONCODE STRING,
    REASONDESCRIPTION STRING,
    FOREIGN KEY (PATIENT) REFERENCES patients (Id)
);"""

sql_table["careplans"] = """CREATE TABLE IF NOT EXISTS careplans (
    Id STRING PRIMARY KEY,
    START DATE,
    STOP DATE,
    PATIENT STRING,
    ENCOUNTER STRING,
    CODE STRING,
    DESCRIPTION STRING,
    REASONCODE STRING,
    REASONDESCRIPTION STRING,
    FOREIGN KEY (PATIENT) REFERENCES patients (Id),
    FOREIGN KEY (ENCOUNTER) REFERENCES encounters (Id)
);"""

sql_table["conditions"] = """CREATE TABLE IF NOT EXISTS conditions (
    START DATE,
    STOP DATE,
    PATIENT STRING,
    ENCOUNTER STRING,
    CODE STRING,
    DESCRIPTION STRING,
    FOREIGN KEY (PATIENT) REFERENCES patients (Id),
    FOREIGN KEY (ENCOUNTER) REFERENCES encounters (Id)
);"""

sql_table["medications"] = """CREATE TABLE IF NOT EXISTS medications (
    START DATE,
    STOP DATE,
    PATIENT STRING,
    PAYER STRING,
    ENCOUNTER STRING,
    CODE STRING,
    DESCRIPTION STRING,
    BASE_COST INTEGER,
    PAYER_COVERAGE INTEGER,
    DISPENSES INTEGER,
    TOTALCOST INTEGER,
    REASONCODE STRING,
    REASONDESCRIPTION STRING,
    FOREIGN KEY (PATIENT) REFERENCES patients (Id)
);"""

sql_table["procedures"] = """CREATE TABLE IF NOT EXISTS procedures (
    DATE DATE,
    PATIENT STRING,
    ENCOUNTER STRING,
    CODE STRING,
    DESCRIPTION STRING,
    BASE_COST INTEGER,
    REASONCODE STRING,
    REASONDESCRIPTION STRING,
    FOREIGN KEY (PATIENT) REFERENCES patients (Id),
    FOREIGN KEY (ENCOUNTER) REFERENCES encounters (Id)
);"""

sql_table["observations"] = """CREATE TABLE IF NOT EXISTS observations (
    DATE DATE,
    PATIENT STRING,
    ENCOUNTER STRING,
    CODE STRING,
    DESCRIPTION STRING,
    VALUE STRING,
    UNITS STRING,
    TYPE STRING,
    FOREIGN KEY (PATIENT) REFERENCES patients (Id),
    FOREIGN KEY (ENCOUNTER) REFERENCES encounters (Id)
);"""

sql_table["devices"] = """CREATE TABLE IF NOT EXISTS devices (
    START DATE,
    STOP DATE,
    PATIENT STRING,
    ENCOUNTER STRING,
    CODE STRING,
    DESCRIPTION STRING,
    UDI STRING,
    FOREIGN KEY (PATIENT) REFERENCES patients (Id),
    FOREIGN KEY (ENCOUNTER) REFERENCES encounters (Id)
);"""

sql_table["imaging_studies"] = """CREATE TABLE IF NOT EXISTS imaging_studies (
    Id STRING PRIMARY KEY,
    DATE DATE,
    PATIENT STRING,
    ENCOUNTER STRING,
    BODYSITE_CODE STRING,
    BODYSITE_DESCRIPTION STRING,
    MODALITY_CODE STRING,
    MODALITY_DESCRIPTION STRING,
    SOP_CODE STRING,
    SOP_DESCRIPTION STRING,
    FOREIGN KEY (PATIENT) REFERENCES patients (Id),
    FOREIGN KEY (ENCOUNTER) REFERENCES encounters (Id)
);"""

sql_table["immunizations"] = """CREATE TABLE IF NOT EXISTS immunizations(
    DATE DATE,
    PATIENT STRING,
    ENCOUNTER STRING,
    CODE STRING,
    DESCRIPTION STRING,
    BASE_COST INTEGER,
    FOREIGN KEY (PATIENT) REFERENCES patients (Id),
    FOREIGN KEY (ENCOUNTER) REFERENCES encounters (Id)
);"""


# ----------------------------------------------------------------------------
# 2) Erstellen Sie die Tabellen in der Datenbank
# ----------------------------------------------------------------------------
for table_name, create_statement in sql_table.items():
    cursor.execute(create_statement)

conn.commit()
print("Tabellen erfolgreich erstellt.")


Tabellen erfolgreich erstellt.


# 3. CSV-Dateien importieren
Hier werden die verfügbaren CSV-Dateien (careplans.csv, conditions.csv, devices.csv, encounters.csv, imaging_studies.csv, immunizations.csv, medications.csv) importiert. Die folgenden Tabellen sind durch die zuvor definierten SQL-Statements abgedeckt:

careplans
conditions
devices
imaging_studies
immunizations
medications


In [40]:
# Diese Zuordnung mappt den Dateinamen zum Tabellennamen und listet die Spalten in der Reihenfolge auf,
# in der sie eingefügt werden sollen.

csv_to_table = {
    "careplans.csv": {
        "table": "careplans",
        "columns": [
            "Id","START","STOP","PATIENT","ENCOUNTER",
            "CODE","DESCRIPTION","REASONCODE","REASONDESCRIPTION"
        ]
    },
    "conditions.csv": {
        "table": "conditions",
        "columns": ["START","STOP","PATIENT","ENCOUNTER","CODE","DESCRIPTION"]
    },
    "devices.csv": {
        "table": "devices",
        "columns": ["START","STOP","PATIENT","ENCOUNTER","CODE","DESCRIPTION","UDI"]
    },
    "imaging_studies.csv": {
        "table": "imaging_studies",
        "columns": [
            "Id","DATE","PATIENT","ENCOUNTER","BODYSITE_CODE","BODYSITE_DESCRIPTION",
            "MODALITY_CODE","MODALITY_DESCRIPTION","SOP_CODE","SOP_DESCRIPTION"
        ]
    },
    "immunizations.csv": {
        "table": "immunizations",
        "columns": ["DATE","PATIENT","ENCOUNTER","CODE","DESCRIPTION","BASE_COST"]
    },
    "medications.csv": {
        "table": "medications",
        "columns": [
            "START","STOP","PATIENT","PAYER","ENCOUNTER","CODE","DESCRIPTION",
            "BASE_COST","PAYER_COVERAGE","DISPENSES","TOTALCOST","REASONCODE","REASONDESCRIPTION"
        ]
    },

      "encounters.csv": {  # NEU HINZUGEFÜGT
        "table": "encounters",
        "columns": [
            "Id", "START", "STOP", "PATIENT", "ORGANIZATION",
            "PROVIDER", "PAYER", "ENCOUNTERCLASS", "CODE",
            "DESCRIPTION", "BASE_ENCOUNTER_COST", "TOTAL_CLAIM_COST",
            "PAYER_COVERAGE", "REASONCODE", "REASONDESCRIPTION"
            ]
        }
}


# Wir durchlaufen alle vorhandenen CSV-Dateien im geklonten Repo-Ordner synthea_data
source_folder = "synthea_data"
for file_name in os.listdir(source_folder):
    # Nur .csv-Dateien betrachten
    if file_name.endswith(".csv") and file_name in csv_to_table:
        table_info = csv_to_table[file_name]
        table_name = table_info["table"]
        columns = table_info["columns"]

        # Pfad zur CSV
        csv_path = os.path.join(source_folder, file_name)

        # SQL-Insert-Statement mit passender Spaltenanzahl erstellen
        placeholders = ",".join(["?"] * len(columns))
        col_names = ",".join(columns)
        insert_sql = f"INSERT OR IGNORE INTO {table_name} ({col_names}) VALUES ({placeholders})"

        print(f"Lade Daten aus {file_name} in Tabelle '{table_name}' ...")

        with open(csv_path, "r", encoding="utf-8") as f:
            reader = csv.DictReader(f)
            to_db = []
            for row in reader:
                # Erstelle die Werte in der richtigen Spaltenreihenfolge
                values = [row[col] for col in columns]
                to_db.append(values)

        # Masseninsertion in die Datenbank
        cursor.executemany(insert_sql, to_db)
        conn.commit()

print("CSV-Import abgeschlossen!")


Lade Daten aus encounters.csv in Tabelle 'encounters' ...
Lade Daten aus conditions.csv in Tabelle 'conditions' ...
Lade Daten aus imaging_studies.csv in Tabelle 'imaging_studies' ...
Lade Daten aus medications.csv in Tabelle 'medications' ...
Lade Daten aus immunizations.csv in Tabelle 'immunizations' ...
Lade Daten aus devices.csv in Tabelle 'devices' ...
Lade Daten aus careplans.csv in Tabelle 'careplans' ...
CSV-Import abgeschlossen!


In [34]:
!drop table encounters;

/bin/bash: line 1: drop: command not found


#

# 4. Erste Zeile jeder Tabelle anzeigen


In [35]:
tables_to_check = [
    "careplans",
    "conditions",
    "devices",
    "imaging_studies",
    "immunizations",
    "medications"
]

for t in tables_to_check:
    print(f"\nErste Zeile aus Tabelle '{t}':")
    cursor.execute(f"SELECT * FROM {t} LIMIT 5")
    row = cursor.fetchone()
    print(row)

# Verbindung schließen (ggf. offen lassen, solange Sie Abfragen ausführen wollen)
#conn.close()



Erste Zeile aus Tabelle 'careplans':
('d2500b8c-e830-433a-8b9d-368d30741520', '2010-01-23', '2012-01-23', '034e9e3b-2def-4559-bb2a-7850888ae060', 'd0c40d10-8d87-447e-836e-99d26ad52ea5', 53950000, 'Respiratory therapy', 10509002, 'Acute bronchitis (disorder)')

Erste Zeile aus Tabelle 'conditions':
('2001-05-01', '', '1d604da9-9a81-4ba9-80c2-de3375d59b40', '8f104aa7-4ca9-4473-885a-bba2437df588', 40055000, 'Chronic sinusitis (disorder)')

Erste Zeile aus Tabelle 'devices':
('2001-07-04T08:42:44Z', '', 'd49f748f-928d-40e8-92c8-73e4c5679711', '2500b8bd-dc98-44ef-a252-22dc4f81d61b', 72506001, 'Implantable defibrillator  device (physical object)', '(01)67677988606464(11)010613(17)260628(10)2882441934(21)7849600052')

Erste Zeile aus Tabelle 'imaging_studies':
('d3e49b38-7634-4416-879d-7bc68bf3e7df', '2014-07-08T15:35:36Z', 'b58731cc-2d8b-4c2d-b327-4cab771af3ef', '3a36836d-da25-4e73-808b-972b669b7e4e', 40983000, 'Arm', 'DX', 'Digital Radiography', '1.2.840.10008.5.1.4.1.1.1.1', 'Digital X-Ra

In [36]:
# Daten Ausgeben
cursor.execute("SELECT * FROM medications")
record = cursor.fetchone()
print(record)

('2010-05-05T00:26:23Z', '2011-04-30T00:26:23Z', '8d4c4326-e9de-4f45-9a4c-f8c36bff89ae', 'b1c428d6-4f07-31e0-90f0-68ffa6ff8c76', '1e0d6b0e-1711-4a25-99f9-b1c700c9b260', 389221, 'Etonogestrel 68 MG Drug Implant', 677.08, 0, 12, 8124.96, '', '')


# Aufgabe

Die Tabelle encounters ist noch nicht vollständig angelegt.

1. Legen Sie die encounters Tabellen entsprechend der obigen Beispielcode an

2. Finden Sie alle Einträge in der Tabelle encounters, bei denen der ENCOUNTERCLASS z.B. urgentcare ist.

In [37]:
#Bitte folgende Code entsprechend anpasssen
import sqlite3

# Verbindung zur SQLite-Datenbank herstellen (bzw. Datei erstellen)
conn = sqlite3.connect("synthea.db")
cursor = conn.cursor()

# SQL-Statement zum Erstellen der encounters-Tabelle
create_encounters_table = """
CREATE TABLE IF NOT EXISTS encounters (
    Id STRING PRIMARY KEY,
    START DATE,
    STOP DATE,
    PATIENT STRING,
    ORGANIZATION STRING,
    PROVIDER STRING,
    PAYER STRING,
    ENCOUNTERCLASS STRING,
    CODE STRING,
    DESCRIPTION STRING,
    BASE_ENCOUNTER_COST INTEGER,
    TOTAL_CLAIM_COST INTEGER,
    PAYER_COVERAGE INTEGER,
    REASONCODE STRING,
    REASONDESCRIPTION STRING,
    FOREIGN KEY (PATIENT) REFERENCES patients (Id)
);
"""

# Tabelle erstellen
cursor.execute(create_encounters_table)

# Änderungen speichern und Verbindung schließen
conn.commit()
#conn.close()



print("Tabelle 'encounters' wurde erfolgreich erstellt.")


Tabelle 'encounters' wurde erfolgreich erstellt.


In [38]:
conn = sqlite3.connect("synthea.db")
cursor = conn.cursor()

cursor.execute("SELECT * FROM encounters WHERE ENCOUNTERCLASS = 'urgentcare'")
results = cursor.fetchall()

print("Anzahl Treffer:", len(results))
for row in results[:5]:  # Nur die ersten 5 ausgeben
    print(row)

Anzahl Treffer: 2373
('04f69a96-6f77-4ca6-95f5-b7a45bb9e476', '2011-05-24T12:58:08Z', '2011-05-24T13:13:08Z', '10339b10-3cd1-4ac3-ac13-ec26728cb592', 'a9afd4c9-8443-3b5a-a486-07c3bb109b3f', '08703819-4d5c-3998-9de3-6ef611285bbc', 'd47b3510-2895-3b70-9897-342d681c769d', 'urgentcare', 702927004, 'Urgent care clinic (procedure)', 129.16, 129.16, 0, '', '')
('ae056eca-c1b1-45e5-870b-d4127c33bfa4', '2019-04-30T12:58:08Z', '2019-04-30T13:13:08Z', '10339b10-3cd1-4ac3-ac13-ec26728cb592', 'a9afd4c9-8443-3b5a-a486-07c3bb109b3f', '08703819-4d5c-3998-9de3-6ef611285bbc', 'd47b3510-2895-3b70-9897-342d681c769d', 'urgentcare', 702927004, 'Urgent care clinic (procedure)', 129.16, 129.16, 0, '', '')
('e20a6a4d-343d-479d-9730-fbc4447364f9', '2017-11-11T06:21:56Z', '2017-11-11T06:36:56Z', 'b1e9b0b9-da6e-4f68-b603-bd896a50ca86', '4e916854-3a59-3f17-bb16-7c406148d822', '93f77048-5bfd-3a59-9ae4-4399fbd77398', '4d71f845-a6a9-3c39-b242-14d25ef86a8d', 'urgentcare', 702927004, 'Urgent care clinic (procedure)', 1

# Neuer Abschnitt